
# 01 — Exploratory Analysis: El Paso Dataset


In [1]:

# --- Configuration ---
from pathlib import Path


DATA_DIR = Path("data")
FILE_GLOB = "**/*.csv"

# Column name hints — the loader will try to auto-detect, but you can override.
DATETIME_COL_HINTS = ["time", "timestamp", "date", "datetime", "fecha_hora", "Fecha_Hora", "Unnamed: 0"]
GHI_COL_HINTS = ["GHI", "ghi", "horiz_radiance", "radiance", "ghi_wm2", "GHI2", "diff_radiance"]
TZ = "America/Bogota"  # change if needed


In [2]:

# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, sys, textwrap

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)


In [3]:

# --- Discover files ---
paths = sorted(DATA_DIR.glob(FILE_GLOB))
print(f"Found {len(paths)} file(s).")
for p in paths[:10]:
    print("•", p)
if len(paths) > 10:
    print("...")
assert len(paths) > 0, "No files found. Check DATA_DIR and FILE_GLOB."


Found 0 file(s).


AssertionError: No files found. Check DATA_DIR and FILE_GLOB.

In [ ]:

# --- Loader with datetime detection & basic cleaning ---
def _guess_column(cols, hints):
    cols_lower = {c.lower(): c for c in cols}
    for h in hints:
        if h.lower() in cols_lower:
            return cols_lower[h.lower()]
    return None

def load_csv(path):
    df = pd.read_csv(path)
    # try datetime column
    dt_col = _guess_column(df.columns, DATETIME_COL_HINTS)
    if dt_col is None:
        # try infer from any column containing 'time' or 'date'
        candidates = [c for c in df.columns if any(k in c.lower() for k in ["time", "date"])]
        dt_col = candidates[0] if candidates else None
    if dt_col is not None:
        df[dt_col] = pd.to_datetime(df[dt_col], errors="coerce")
        # If tz-naive, localize; if tz-aware, convert
        if pd.api.types.is_datetime64_any_dtype(df[dt_col]):
            if df[dt_col].dt.tz is None:
                df[dt_col] = df[dt_col].dt.tz_localize(TZ, nonexistent="shift_forward", ambiguous="NaT")
            else:
                df[dt_col] = df[dt_col].dt.tz_convert(TZ)
        df = df.sort_values(dt_col).dropna(subset=[dt_col])
        df = df.set_index(dt_col)
    return df

dfs = []
for p in paths:
    try:
        df = load_csv(p)
        df["__source_file"] = str(p)
        dfs.append(df)
    except Exception as e:
        print(f"[WARN] Failed to load {p}: {e}")

assert len(dfs) > 0, "No CSVs could be loaded. Inspect formats and update loader if needed."


In [ ]:

# --- Concatenate and quick overview ---
data = pd.concat(dfs, axis=0, sort=False)
data = data[~data.index.duplicated(keep="first")]
print(f"Shape: {data.shape}")
display(data.head(3))
display(data.tail(3))

print("\nColumns:\n", list(data.columns))
print("\nTime range:", data.index.min(), "→", data.index.max())
print("Frequency (approx.):", pd.infer_freq(data.index[:500]) or "irregular / unknown")


In [ ]:

# --- Missing values summary ---
na_counts = data.isna().sum().sort_values(ascending=False)
na_ratio = (na_counts / len(data)).round(4)
missing_report = pd.DataFrame({"missing": na_counts, "ratio": na_ratio})
display(missing_report.head(20))


In [ ]:

# --- Identify GHI-like column and basic stats ---
def find_ghi_col(columns):
    for h in GHI_COL_HINTS:
        if h in columns:
            return h
    # fallback: look for 'ghi' substring
    for c in columns:
        if "ghi" in c.lower():
            return c
    return None

ghi_col = find_ghi_col(list(data.columns))
print("Detected GHI column:", ghi_col)

if ghi_col is not None:
    ghi = data[ghi_col].astype("float64")
    print(ghi.describe(percentiles=[0.01, 0.1, 0.5, 0.9, 0.99]))
else:
    print("No GHI-like column detected. You can set ghi_col manually.")


In [ ]:

# --- Daily profile (if GHI available) ---
if ghi_col is not None:
    # drop extreme outliers for plotting
    s = data[ghi_col].astype("float64").clip(lower=0)
    daily = s.groupby([s.index.tz_convert(TZ).hour]).median()
    plt.figure(figsize=(8,4))
    daily.plot(kind="line")
    plt.title("Median daily profile (hour of day)")
    plt.xlabel("Hour")
    plt.ylabel(f"{ghi_col}")
    plt.grid(True)
    plt.show()


In [ ]:

# --- Resampling helpers ---
def resample_series(series, rule="10T", how="mean"):
    if how == "mean":
        return series.resample(rule).mean()
    if how == "median":
        return series.resample(rule).median()
    if how == "max":
        return series.resample(rule).max()
    if how == "min":
        return series.resample(rule).min()
    raise ValueError("Unsupported aggregation")

if ghi_col is not None:
    for rule in ["1T", "10T", "60T", "1D"]:
        rs = resample_series(data[ghi_col].astype("float64"), rule=rule, how="mean")
        plt.figure(figsize=(8,3))
        rs.plot()
        plt.title(f"{ghi_col} — Resampled to {rule}")
        plt.xlabel("Time")
        plt.ylabel(ghi_col)
        plt.grid(True)
        plt.show()


In [ ]:

# --- Quick correlation heatmap (numeric columns) ---
num = data.select_dtypes(include=[np.number])
if len(num.columns) >= 2:
    corr = num.corr()
    plt.figure(figsize=(6,5))
    plt.imshow(corr, aspect="auto")
    plt.colorbar(label="Correlation")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.title("Correlation matrix (numeric columns)")
    plt.tight_layout()
    plt.show()
else:
    print("Not enough numeric columns for a correlation heatmap.")


In [ ]:

# --- Gap analysis (time index) ---
idx = data.index
if hasattr(idx, "asi8"):
    diffs = pd.Series(idx.asi8[1:] - idx.asi8[:-1]).astype("float64") / 1e9
    plt.figure(figsize=(8,3))
    plt.plot(diffs.values)
    plt.title("Index time gaps (seconds between consecutive samples)")
    plt.xlabel("Step")
    plt.ylabel("Δt (s)")
    plt.grid(True)
    plt.show()
else:
    print("Index is not datetime-like; gap analysis skipped.")
